In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import  Dataset,random_split,DataLoader

In [ ]:
# 定义一个名为 plot_series 的函数，用于绘制时间序列图
def plot_series(time, plot_series, format='-', start=0, end=None, label=None):
    # 使用 matplotlib 绘制时间序列图
    plt.plot(time[start:end], plot_series[start:end], format, label=label)
    plt.xlabel("Time")  # x轴标签
    plt.ylabel("Value")  # y轴标签
    if label:  # 如果提供了标签
        plt.legend(fontsize=14)  # 显示图例
    plt.grid(True)  # 显示网格
    plt.show()  # 显示图像


In [ ]:
# 定义名为 trend 的函数，用于生成趋势数据
def trend(time, slope=0):
    """
    生成趋势数据。

    参数:
    time (list): 时间的列表。
    slope (float): 坡度，默认为0。
    """
    return (slope * time)

# 定义名为 whtie_noise 的函数，用于生成白噪声数据
def whtie_noise(time, std=1.0, seed=None):
    """
    生成白噪声数据。

    参数:
    time (list): 时间的列表。
    std (float): 标准差，默认为1.0。
    seed (int): 种子数，默认为None。
    """
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * std

# 定义名为 seasonal_pattern 的函数，用于生成季节性模式数据
def seasonal_pattern(season_time):
    """
    生成季节性模式数据。

    参数:
    season_time (list): 季节时间的列表。
    """
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

# 定义名为 seasonality 的函数，用于生成季节性数据
def seasonality(time, period, amplitude=1, phase=0):
    """
    生成季节性数据。

    参数:
    time (list): 时间的列表。
    period (int): 周期数。
    amplitude (int): 振幅，默认为1。
    phase (int): 相位，默认为0。
    """
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)
def train_test_split(series, split_fraction=0.7):
    """
    将数据集划分为训练集和测试集。

    参数:
    series (list): 输入的时间序列数据。
    split_fraction (float): 划分的比例，默认为0.7。

    返回:
    train_set (list): 训练集数据。
    test_set (list): 测试集数据。
    """
    train_size = int(len(series) * split_fraction)
    train_set = series[:train_size]
    test_set = series[train_size:]
    return train_set, test_set


In [ ]:

time = np.arange(4 * 365 + 1)
baseline = 10
slope = 0.05
amplitude = 40
noise_level = 2

# 生成时间序列数据
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude) + whtie_noise(time, noise_level, seed=42)
series = series.astype(np.double)

# 绘制时间序列图
plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()


In [ ]:
import torch
import random
from torch.utils.data import Dataset, DataLoader

split_prop = 0.7  # 划分比例
train, test = train_test_split(series, split_prop)  # 划分训练集和测试集